In [1]:
from __future__ import print_function

import numpy as np
import time
from keras.preprocessing.image import save_img
from keras.applications import inception_v3
from keras import backend as K

# dimensions of the generated pictures for each filter.
img_width = 128
img_height = 128

Using TensorFlow backend.


In [2]:
# util function to convert a tensor into a valid image
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [3]:

model = inception_v3.InceptionV3(include_top=False,weights='imagenet')
print('model loaded')

model loaded


In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [5]:
# this is the placeholder for the input images
input_img = model.input

In [6]:

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'activation_94'
print(layer_dict[layer_name])

In [7]:
def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())


In [8]:
kept_filters = []
for filter_index in range(200):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))


Processing filter 0
Current loss value: 16.474705
Current loss value: 74.17623
Current loss value: 109.6045
Current loss value: 145.6452
Current loss value: 169.97594
Current loss value: 198.21436
Current loss value: 241.28331
Current loss value: 269.17502
Current loss value: 298.92993
Current loss value: 337.1813
Current loss value: 366.90527
Current loss value: 409.5593
Current loss value: 444.45963
Current loss value: 472.7512
Current loss value: 495.36215
Current loss value: 505.07642
Current loss value: 532.62946
Current loss value: 554.2102
Current loss value: 590.86536
Current loss value: 609.76483
Filter 0 processed in 5s
Processing filter 1
Current loss value: 56.311527
Current loss value: 113.63225
Current loss value: 154.83752
Current loss value: 187.44359
Current loss value: 247.26141
Current loss value: 277.34167
Current loss value: 313.63834
Current loss value: 336.5029
Current loss value: 362.7242
Current loss value: 410.89224
Current loss value: 453.0484
Current loss va

Current loss value: 845.4257
Current loss value: 917.26373
Current loss value: 989.36835
Current loss value: 1046.9431
Filter 21 processed in 7s
Processing filter 22
Current loss value: 56.72976
Current loss value: 111.66561
Current loss value: 148.40736
Current loss value: 196.43314
Current loss value: 245.8677
Current loss value: 276.01328
Current loss value: 313.52792
Current loss value: 349.58386
Current loss value: 367.94675
Current loss value: 399.80396
Current loss value: 433.96793
Current loss value: 439.40976
Current loss value: 472.47794
Current loss value: 426.09256
Current loss value: 505.96515
Current loss value: 462.32928
Current loss value: 553.1442
Current loss value: 548.0912
Current loss value: 598.45465
Current loss value: 576.9035
Filter 22 processed in 7s
Processing filter 23
Current loss value: 0.0
Filter 23 processed in 5s
Processing filter 24
Current loss value: 0.0
Filter 24 processed in 5s
Processing filter 25
Current loss value: 42.28989
Current loss value: 8

Current loss value: 82.51004
Current loss value: 118.79058
Current loss value: 152.33417
Current loss value: 169.13701
Current loss value: 194.32904
Current loss value: 222.94429
Current loss value: 247.63737
Current loss value: 274.50723
Current loss value: 300.7542
Current loss value: 271.23877
Current loss value: 327.53073
Current loss value: 333.32367
Current loss value: 354.85666
Current loss value: 318.58066
Current loss value: 384.99814
Current loss value: 402.08984
Current loss value: 402.1455
Current loss value: 375.37332
Current loss value: 434.81525
Current loss value: 389.19998
Filter 47 processed in 12s
Processing filter 48
Current loss value: 77.43676
Current loss value: 120.73792
Current loss value: 153.4166
Current loss value: 194.05652
Current loss value: 229.99329
Current loss value: 259.60135
Current loss value: 251.62807
Current loss value: 307.36783
Current loss value: 291.6623
Current loss value: 338.0337
Current loss value: 350.71164
Current loss value: 391.57047

Current loss value: 0.0
Filter 66 processed in 192s
Processing filter 67
Current loss value: 0.0
Filter 67 processed in 189s
Processing filter 68
Current loss value: 95.017845
Current loss value: 154.89677
Current loss value: 222.40846
Current loss value: 271.98935
Current loss value: 322.71863
Current loss value: 357.62064
Current loss value: 396.28998
Current loss value: 426.5182
Current loss value: 445.028
Current loss value: 490.5657
Current loss value: 529.3679
Current loss value: 530.3849
Current loss value: 547.3529
Current loss value: 535.46375
Current loss value: 609.47565
Current loss value: 570.8121
Current loss value: 656.65686
Current loss value: 634.9555
Current loss value: 686.82446
Current loss value: 624.18176
Filter 68 processed in 149s
Processing filter 69
Current loss value: 116.209595
Current loss value: 170.7856
Current loss value: 212.1412
Current loss value: 246.91621
Current loss value: 249.91345
Current loss value: 219.63097
Current loss value: 279.39877
Curre

Current loss value: 0.0
Filter 94 processed in 25s
Processing filter 95
Current loss value: 179.54373
Current loss value: 229.87975
Current loss value: 266.8705
Current loss value: 296.08197
Current loss value: 341.25897
Current loss value: 388.273
Current loss value: 419.09384
Current loss value: 453.1794
Current loss value: 474.8528
Current loss value: 501.97922
Current loss value: 515.8156
Current loss value: 543.98535
Current loss value: 572.5217
Current loss value: 581.78674
Current loss value: 632.21075
Current loss value: 641.87695
Current loss value: 672.8954
Current loss value: 652.7241
Current loss value: 707.6479
Current loss value: 664.674
Filter 95 processed in 29s
Processing filter 96
Current loss value: 0.0
Filter 96 processed in 26s
Processing filter 97
Current loss value: 0.0
Filter 97 processed in 27s
Processing filter 98
Current loss value: 86.67437
Current loss value: 166.44048
Current loss value: 235.13422
Current loss value: 285.6938
Current loss value: 355.509
Cu

Current loss value: 318.2322
Current loss value: 248.71498
Current loss value: 340.5114
Current loss value: 343.30124
Current loss value: 337.898
Current loss value: 313.6355
Current loss value: 365.29514
Current loss value: 349.00177
Filter 116 processed in 39s
Processing filter 117
Current loss value: 231.40689
Current loss value: 292.94733
Current loss value: 334.85593
Current loss value: 380.248
Current loss value: 427.207
Current loss value: 474.04578
Current loss value: 529.7491
Current loss value: 591.5555
Current loss value: 639.39545
Current loss value: 690.8031
Current loss value: 746.8359
Current loss value: 782.26587
Current loss value: 811.92017
Current loss value: 821.49207
Current loss value: 880.7424
Current loss value: 814.51105
Current loss value: 936.73987
Current loss value: 958.11646
Current loss value: 987.38367
Current loss value: 1018.29285
Filter 117 processed in 40s
Processing filter 118
Current loss value: 1.9616426
Current loss value: 45.101055
Current loss 

Current loss value: 331.90958
Current loss value: 332.7804
Filter 140 processed in 55s
Processing filter 141
Current loss value: 120.20559
Current loss value: 154.40169
Current loss value: 184.39679
Current loss value: 220.11449
Current loss value: 250.16907
Current loss value: 258.90396
Current loss value: 282.89102
Current loss value: 301.96558
Current loss value: 317.9854
Current loss value: 347.03195
Current loss value: 371.8155
Current loss value: 401.78787
Current loss value: 420.1579
Current loss value: 437.8661
Current loss value: 462.7645
Current loss value: 433.17905
Current loss value: 497.1407
Current loss value: 521.5351
Current loss value: 534.8676
Current loss value: 563.05725
Filter 141 processed in 58s
Processing filter 142
Current loss value: 73.20593
Current loss value: 123.578285
Current loss value: 180.61801
Current loss value: 232.84274
Current loss value: 294.75626
Current loss value: 332.41214
Current loss value: 373.65308
Current loss value: 415.04834
Current l

Current loss value: 359.44904
Current loss value: 373.42093
Current loss value: 394.50742
Current loss value: 407.85068
Current loss value: 420.1857
Current loss value: 442.8701
Filter 160 processed in 73s
Processing filter 161
Current loss value: 0.0
Filter 161 processed in 68s
Processing filter 162
Current loss value: 0.0
Filter 162 processed in 71s
Processing filter 163
Current loss value: 0.0
Filter 163 processed in 71s
Processing filter 164
Current loss value: 167.49823
Current loss value: 210.1942
Current loss value: 246.5052
Current loss value: 291.1174
Current loss value: 335.3228
Current loss value: 375.64792
Current loss value: 405.0337
Current loss value: 434.7218
Current loss value: 459.86478
Current loss value: 474.18494
Current loss value: 478.9291
Current loss value: 499.74268
Current loss value: 512.26483
Current loss value: 553.0912
Current loss value: 575.11053
Current loss value: 602.41785
Current loss value: 643.9313
Current loss value: 670.1152
Current loss value: 

Current loss value: 0.0
Filter 185 processed in 92s
Processing filter 186
Current loss value: 0.0
Filter 186 processed in 98s
Processing filter 187
Current loss value: 0.0
Filter 187 processed in 94s
Processing filter 188
Current loss value: 130.7109
Current loss value: 169.07202
Current loss value: 190.12549
Current loss value: 190.989
Current loss value: 219.20735
Current loss value: 238.52376
Current loss value: 250.94946
Current loss value: 266.37885
Current loss value: 277.6544
Current loss value: 296.01962
Current loss value: 323.48026
Current loss value: 339.66354
Current loss value: 335.83252
Current loss value: 321.06445
Current loss value: 384.04654
Current loss value: 406.05667
Current loss value: 435.51135
Current loss value: 456.7121
Current loss value: 489.13202
Current loss value: 511.38248
Filter 188 processed in 105s
Processing filter 189
Current loss value: 0.0
Filter 189 processed in 104s
Processing filter 190
Current loss value: 0.0
Filter 190 processed in 97s
Proce

ValueError: slice index 192 of dimension 3 out of bounds. for 'strided_slice_192' (op: 'StridedSlice') with input shapes: [?,?,?,192], [4], [4], [4] and with computed input tensors: input[1] = <0 0 0 192>, input[2] = <0 0 0 193>, input[3] = <1 1 1 1>.

In [ ]:
# we will stich the best 64 filters on a 8 x 8 grid.
n = 8

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]


In [ ]:
# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))


In [ ]:
# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                         (img_height + margin) * j: (img_height + margin) * j + img_height, :] = img


In [ ]:
# save the result to disk
save_img('stitched_filters_%dx%d.png' % (n, n), stitched_filters)

In [ ]:
%matplotlib inline
from matplotlib.pyplot import imshow

imshow(stitched_filters)